<a href="https://colab.research.google.com/github/A-braxas/Dive-into-deep-learning/blob/main/%E5%8A%A8%E6%89%8B%E5%AD%A6%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 配置环境

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Dive into deep learning")

In [ ]:
!pwd

/content/gdrive/My Drive/Colab Notebooks/Dive into deep learning


In [ ]:
!pip install -q mxnet-cu101

     |███████████████████████████████▌| 834.1MB 1.9MB/s eta 0:00:07tcmalloc: large alloc 1147494400 bytes == 0x55be1d23e000 @  0x7f256a2dc615 0x55bde374506c 0x55bde3824eba 0x55bde3747e8d 0x55bde383999d 0x55bde37bbfe9 0x55bde37b6b0e 0x55bde374977a 0x55bde37bbe50 0x55bde37b6b0e 0x55bde374977a 0x55bde37b886a 0x55bde383a7c6 0x55bde37b7ee2 0x55bde383a7c6 0x55bde37b7ee2 0x55bde383a7c6 0x55bde37b7ee2 0x55bde383a7c6 0x55bde37b7ee2 0x55bde374969a 0x55bde37b7c9e 0x55bde37b6e0d 0x55bde374977a 0x55bde37b7a45 0x55bde374969a 0x55bde37b7a45 0x55bde37b6b0e 0x55bde374977a 0x55bde37b886a 0x55bde37b6b0e
     |████████████████████████████████| 846.4MB 19kB/s 


In [ ]:
import mxnet as mx
from mxnet import nd, autograd, gluon

In [ ]:
a = mx.nd.ones((2, 3))
b = a * 2 + 1
b.asnumpy()

# 程序

In [ ]:
from IPython import display
from matplotlib import pyplot as plt
from mxnet import autograd, nd
import random

In [ ]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += nd.random.normal(scale=0.01, shape=labels.shape)

In [27]:
features[0], labels[0]

(
 [2.2122064 0.7740038]
 <NDArray 2 @cpu(0)>, 
 [6.000587]
 <NDArray 1 @cpu(0)>)